In [59]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models
import os
from PIL import Image

In [60]:
# Define the CNN model architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 1)  # Output layer for binary classification (real vs. spoofed)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

In [61]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.classes = os.listdir(root_dir)
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.transform = transform
        self.data = self._load_data()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.float32) / (len(self.classes) - 1)  # Normalize label to [0, 1]
        return image, label

    def _load_data(self):
        data = []
        for cls in self.classes:
            cls_dir = os.path.join(self.root_dir, cls)
            if os.path.isdir(cls_dir):
                images = os.listdir(cls_dir)
                for img in images:
                    img_path = os.path.join(cls_dir, img)
                    data.append((img_path, self.class_to_idx[cls]))
        return data

In [62]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [63]:
# Set device (GPU if available, else CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [64]:
# Define hyperparameters
batch_size = 1
learning_rate = 0.001
num_epochs = 10

In [65]:
# Define dataset paths
train_dir = "dataset/students/KRS Door automation images"
# train_dir = "KRS Door automation images/train"
# val_dir = "KRS Door automation images/val"
# test_dir = "KRS Door automation images/test"

# Create datasets
train_dataset = CustomDataset(train_dir, transform=transform)
# val_dataset = CustomDataset(val_dir, transform=transform)
# test_dataset = CustomDataset(test_dir, transform=transform)

# Create data loaders
batch_size = 1
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [66]:
# Check the number of classes
num_classes = len(train_dataset.classes)
print("Number of classes:", num_classes)

Number of classes: 42


In [67]:
# Initialize model, loss function, and optimizer
model = CNNModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [68]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch + 1}, Training Loss: {running_loss / len(train_loader)}")

Epoch 1, Training Loss: 0.6958469762549185


In [ ]:
# # Evaluation on validation set
# model.eval()
# val_correct = 0
# val_total = 0
# with torch.no_grad():
#     for images, labels in val_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         predicted = torch.round(outputs)
#         val_correct += (predicted == labels.float().unsqueeze(1)).sum().item()
#         val_total += labels.size(0)
# val_accuracy = val_correct / val_total
# print(f"Validation Accuracy: {val_accuracy}")

# # Evaluation on test set
# test_correct = 0
# test_total = 0
# with torch.no_grad():
#     for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         predicted = torch.round(outputs)
#         test_correct += (predicted == labels.float().unsqueeze(1)).sum().item()
#         test_total += labels.size(0)
# test_accuracy = test_correct / test_total
# print(f"Test Accuracy: {test_accuracy}")